## 대회 데이터셋 구성

### data
    ├── directors.tsv
    ├── genres.tsv
    ├── titles.tsv
    ├── train_ratings.csv
    ├── writers.tsv
    └── years.tsv
    
1. *train_ratings.csv* : 전체 훈련 데이터.
2. *directors.tsv*, *genres.tsv*, *writers.tsv*, *years.tsv*, *titles.tsv* : 영화의 side-information 데이터.

### train/test 데이터셋

train 데이터셋은 *user, item, time* 세개의 column으로 구성된 반면, test 데이터셋은 *user* column과 비어있는 *item* column으로 구성되어 있음을 참고해 주세요. 대회 진행시, 각각의 사용자에게 10개의 영화를 추천하게 됩니다.

각 column은 다음을 나타냅니다.
- *user*: 사용자 id.
- *item*: 영화 id.
- *time*: 사용자가 해당 영화와 interact한 시간. (UNIX시간의 초 단위)

영화 id는 *directors.tsv, genres.tsv, writers.tsv, years.tsv, titles.tsv*에서도 일관적으로 사용됩니다. 즉, 동일한 영화 id는 side information 데이터셋에서 동일한 영화를 나타냅니다.

## 라이브러리 불러오기

In [ ]:
import os
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## 데이터 불러오기

### 1. train

In [ ]:
data_path = "~/movie/data/train"
train_df = pd.read_csv(os.path.join(data_path, "train_ratings.csv")) # 전체 학습 데이터

In [ ]:
# train 데이터 데이터 타입, 결측치 확인
train_df.info()
train_df

In [ ]:
# train 데이터 사용자, 아이템 unique 수 확인
num_train_users = train_df["user"].nunique()
num_train_items = train_df["item"].nunique()

print ("Number of unique train users: ", num_train_users)
print ("Number of unique train items: ", num_train_items)
print("Data sparsity ratio: ", 1 - len(train_df) / (num_train_users * num_train_items))

데이터의 sparsity ratio는 약 97.6%로 일반적인 추천 시스템 데이터셋에 비해 굉장히 dense한 (행렬 상 이력의 많은 부분이 채워져 있는) 데이터임을 알 수 있습니다.

In [ ]:
# 유저별 시청 이력 수 기술통계량 확인
tmp = pd.DataFrame(train_df["user"].value_counts())
tmp.describe()

최소 16편에서 최대 2912편까지 유저별 시청 이력이 존재하고, 유저 평균 시청 횟수는 164편의 영화를 시청했음을 확인할 수 있다.

In [ ]:
# 유저별 시청 횟수 시각화: 상자 그림, 히스토그램
fig, axes = plt.subplots(1, 2, figsize=(12, 5))
sns.boxplot(tmp, ax=axes[0])
sns.histplot(tmp, ax=axes[1])
plt.show()

In [ ]:
# 영화별 시청 횟수 기술통계량 확인
tmp = pd.DataFrame(train_df["item"].value_counts())
tmp.describe()

최소 27번에서 최대 19,699번까지 영화 시청 수가 존재하고, 영화 평균 시청 횟수는 757번 시청됐음을 확인할 수 있다.

In [ ]:
# 유저별 시청 횟수 시각화: 상자 그림, 히스토그램
fig, axes = plt.subplots(1, 2, figsize=(12, 5))
sns.boxplot(tmp, ax=axes[0])
sns.histplot(tmp, ax=axes[1])
plt.show()

### 2. side information

In [ ]:
# 아이템 side information 불러오기
year_data = pd.read_csv(os.path.join(data_path, 'years.tsv'), sep='\t')
writer_data = pd.read_csv(os.path.join(data_path, 'writers.tsv'), sep='\t')
title_data = pd.read_csv(os.path.join(data_path, 'titles.tsv'), sep='\t')
genre_data = pd.read_csv(os.path.join(data_path, 'genres.tsv'), sep='\t')
director_data = pd.read_csv(os.path.join(data_path, 'directors.tsv'), sep='\t')

In [ ]:
# side information을 하나의 아이템 데이터프레임으로 병합
item_df = pd.merge(title_data, year_data, on="item", how="left")
item_df = pd.merge(item_df, director_data, on="item", how="left")
item_df = pd.merge(item_df, writer_data, on="item", how="left")
item_df = pd.merge(item_df, genre_data, on="item", how="left")

item_df

In [ ]:
# item_df 데이터 타입, 결측치, 기술통계량 확인
item_df.info()
item_df.describe(include="all")

#### 결측치 확인

In [ ]:
# item_df 결측치 수 확인
# 아이템별로 감독, 작가, 장르가 여러 개인 경우가 있으므로 unique로 정확한 결측치 수를 파악해봐야 한다.
item_df.isna().sum()

In [ ]:
# 결측치가 존재하는 컬럼(year, director, writer)별 unique 결측치 수 확인
print("Number of all unique year missing values :", item_df[item_df["year"].isna()]["title"].nunique())
print("Number of all unique director missing values :", item_df[item_df["director"].isna()]["title"].nunique())
print("Number of all unique writer missing values :", item_df[item_df["writer"].isna()]["title"].nunique())

In [ ]:
# 결측치가 존재하는 컬럼(year, director, writer)별 결측치 확인을 위한 데이터프레임 출력
print("Checking a dataframe for missing values in year data")
display(item_df[item_df["year"].isna()])
print("\nChecking a dataframe for missing values in director data")
display(item_df[item_df["director"].isna()])
print("\nChecking a dataframe for missing values in writer data")
display(item_df[item_df["writer"].isna()])


#### unique 수 확인

In [ ]:
print("Number of unique items:", item_df["item"].nunique())
print("Number of unique titles:", item_df["title"].nunique())
print("Number of unique years:", item_df["year"].nunique())
print("Number of unique directors:", item_df["director"].nunique())
print("Number of unique writers:", item_df["writer"].nunique())
print("Number of unique genres:", item_df["genre"].nunique())

특이사항으로 item id 수와 title 수는 같아야 하는데 1개 차이가 나는 것을 확인할 수 있다. 

어디에서 생긴 문제인지 파악해보자

In [ ]:
# unique item과 unique title 사이의 하나 차이나는 데이터 확인
title_data["title"].value_counts()

In [ ]:
title_data[title_data["title"] == "War of the Worlds (2005)"]

In [ ]:
pd.concat((item_df[item_df["item"] == 34048], item_df[item_df["item"] == 64997]))

스티븐 스필버그 감독의 영화 우주전쟁(2005)에서 director 컬럼에 결측치가 있는 경우 다른 item id로 분류하여 item과 title의 unique 수에서 1개가 차이가 발생함을 확인할 수 있다.

#### 데이터 시각화

In [ ]:
# 연도별 영화 수 확인을 위해 item, year 컬럼 중복 제거
tmp = item_df.drop_duplicates(["item", "year"])
tmp

In [ ]:
# 연도별 히스토그램 시각화
sns.histplot(data=tmp, x="year", kde=True)
plt.show()

In [ ]:
# 장르별 영화 수 확인을 위해 item, genre 컬럼 중복 제거
tmp = item_df.drop_duplicates(["item", "genre"])
tmp

In [ ]:
# 장르별 영화 수 카운트
tmp["genre"].value_counts()

In [ ]:
# 장르별 히스토그램 시각화
sns.barplot(tmp["genre"].value_counts(), orient="h")
plt.show()

In [ ]:
tmp = item_df.drop_duplicates(["item", "director", "genre"])
tmp

In [ ]:
fig, ax = plt.subplots(figsize=(20, 7))
sns.boxplot(data=tmp, x="genre", y="year", palette="viridis")
plt.show()

### 3. train, item 병합

In [ ]:
merged_df = pd.merge(train_df, item_df, on="item", how="left")
merged_df

In [ ]:
# 유닉스 타임으로 이루어진 time을 pandas의 datetime 객체로 변환
# merged_df["time"] = merged_df["time"].apply(lambda x: time.strftime("%Y-%m", time.localtime(x)))  # 벡터화로 변환
merged_df["time"] = pd.to_datetime(merged_df["time"], unit="s").astype(str)
merged_df

In [ ]:
merged_df.drop_duplicates(["user", "item"])

In [ ]:
merged_df.drop_duplicates(["user", "item", "time"])

shape이 변하지 않는 것으로 보아 같은 영화를 두 번 이상 시청한 이력이 나타나지 않는 것으로 확인된다.

In [ ]:
tmp = merged_df[merged_df["user"] == 11].drop_duplicates(["user", "item", "time"])
tmp[tmp["time"] == "2009-01-01 05:26:00"]

# sns.histplot()
# plt.show()

대신 같은 시간대에 한 유저가 2편 이상 시청 이력이 존재하기도 한다.